# Process Messed Up Jsons

In [31]:
import os
import json
import re
import pandas as pd

In [32]:
def Other_Process_One_V2_Fight_Details(jsn):
    # Split the data into lists for easier management
    list_f1_strikes = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][0]['Fighter'][0]['Strikes']
    list_f1_grappling = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][0]['Fighter'][1]['Grappling']
    list_f1_tip = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][0]['Fighter'][2]['TIP']
    list_f2_strikes = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][1]['Fighter'][0]['Strikes']
    list_f2_grappling = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][1]['Fighter'][1]['Grappling']
    list_f2_tip = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][1]['Fighter'][2]['TIP']  

    # Create new clean dictionary
    clean_dict = {}

    # Process each dataset
    clean_dict = parse_list_strikes_tip(clean_dict, list_f1_strikes, 'F1')
    clean_dict = parse_list_grappling(clean_dict, list_f1_grappling, 'F1')
    clean_dict = parse_list_strikes_tip(clean_dict, list_f1_tip, 'F1')
    clean_dict = parse_list_strikes_tip(clean_dict, list_f2_strikes, 'F2')
    clean_dict = parse_list_grappling(clean_dict, list_f2_grappling, 'F2')
    clean_dict = parse_list_strikes_tip(clean_dict, list_f2_tip, 'F2')
    clean_dict = get_other_keys(clean_dict, jsn)
    clean_dict = add_IDs_and_names(clean_dict, jsn)
    
    return clean_dict

In [33]:
def parse_list_grappling(clean_dict, list_dicts, fighter_string): 
    for one_dict in list_dicts:
        
        # Ignore any dictionary for kicks and punches (since always null)
        if 'kick' in one_dict['Name'].lower() or 'punch' in one_dict['Name'].lower():
            continue

        # Put the values into the new dictionary
        if 'Attempts' in one_dict:
            attempts_key = f"{fighter_string}_Grappling_{re.sub(' ','_',one_dict['Name'].strip())}_Attempts"
            clean_dict[attempts_key] = try_int(one_dict['Attempts'])

        if 'Landed' in one_dict:
            landed_key = f"{fighter_string}_Grappling_{re.sub(' ','_',one_dict['Name'].strip())}_Landed"
            clean_dict[landed_key] = try_int(one_dict['Landed'])

        if 'Success' in one_dict:
            landed_key = f"{fighter_string}_Grappling_{re.sub(' ','_',one_dict['Name'].strip())}_Landed"
            clean_dict[landed_key] = try_int(one_dict['Success'])  
        
        if 'Time' in one_dict:
            tip_key = f"{fighter_string}_TIP_{re.sub(' ','_',one_dict['Name'].strip())}"
            clean_dict[tip_key] = one_dict['Time']
            
    return clean_dict

In [48]:
def parse_list_strikes_tip(clean_dict, list_dicts, fighter_string): 
    for one_dict in list_dicts:
        
        # Ignore any dictionary for kicks and punches (since always null)
        if 'kick' in one_dict['Name'].lower() or 'punch' in one_dict['Name'].lower():
            continue

        # Put the values into the new dictionary
        if 'Attempts' in one_dict:
            attempts_key = f"{fighter_string}_{re.sub(' ','_',one_dict['Name'].strip())}_Attempts"
            try: 
                val = int(one_dict['Attempts']) 
            except: 
                val = one_dict['Attempts']
            clean_dict[attempts_key] = one_dict['Attempts']

        if 'Landed' in one_dict:
            landed_key = f"{fighter_string}_{re.sub(' ','_',one_dict['Name'].strip())}_Landed"
            clean_dict[landed_key] = one_dict['Landed']

        if 'Success' in one_dict:
            landed_key = f"{fighter_string}_{re.sub(' ','_',one_dict['Name'].strip())}_Landed"
            clean_dict[landed_key] = one_dict['Success']  
        
        if 'Time' in one_dict:
            tip_key = f"{fighter_string}_TIP_{re.sub(' ','_',one_dict['Name'].strip())}"
            clean_dict[tip_key] = one_dict['Time']
            
    return clean_dict

SyntaxError: invalid syntax (<ipython-input-48-fb5053bb2d98>, line 11)

In [35]:
def get_other_keys(clean_dict, jsn): 
    Other_Columns_Map = {'FMLiveFeed_FightCard_Accolade':'Accolade',
     'FMLiveFeed_FightCard_CurRound':'CurrentRnd',
     'Missing1':'CurrentRndTime',
     'FMLiveFeed_EventID':'EventID',
     'FMLiveFeed_FightCard_FightID':'FightID',
     'FMLiveFeed_FightCard_TotalNumOfRounds':'MaxRnds',
     'FMLiveFeed_FightCard_Referee':'Referee',
     'FMLiveFeed_FightCard_Status':'Status',
     'Timestamp':'Timestamp',
     'FMLiveFeed_FightCard_WeightClass':'WeightClass'}

    # Flatten the file 
    flat_jsn = flatten_json(jsn)
    
    # delete unwanted keys
    keys_to_delete = []
    for key in flat_jsn.keys():
        if 'RoundStats' in key or 'FightStats' in key or 'Fighters' in key:
            keys_to_delete.append(key)
    for key in keys_to_delete:
        del flat_jsn[key]
        
     # Append to clean_dict
    for key, value in flat_jsn.items():
        if key in Other_Columns_Map:
            clean_dict[Other_Columns_Map[key]] = value   
    
    return clean_dict

In [36]:
def add_IDs_and_names(clean_dict, jsn):
    name_dict = {}
    for f_dict in jsn['FMLiveFeed']['FightCard']['Fight'][0]['Fighters']:
        name_dict[f_dict['FighterID']] = f_dict['Name']
        
    f1_fighterID = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][0]['FighterID']
    f2_fighterID = jsn['FMLiveFeed']['FightCard']['Fight'][2]['FightStats'][1]['FighterID']
    
    #Add FighterIDs and Names to dictionary
    clean_dict['F1_FighterID'] = f1_fighterID
    clean_dict['F2_FighterID'] = f2_fighterID
    clean_dict['F1_Name'] = name_dict[f1_fighterID]
    clean_dict['F2_Name'] = name_dict[f2_fighterID]
    
    return clean_dict

In [49]:
def try_int(value):
    try:
        return int(value)
         
    except:
        return value

In [50]:
try_int('9')

9

In [51]:
try_int('9dif')

'9dif'

In [37]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

## Quick test with 1 JSON

In [38]:
filepath = '../../02_Data/01_Raw_Scraped_Data/V2_Jsons/event_759/fight_5930.json'
with open(filepath) as f:
    jsn = json.load(f)

In [41]:
clean_dict = Other_Process_One_V2_Fight_Details(jsn)

In [44]:
clean_dict

{'Accolade': '',
 'CurrentRnd': '2',
 'EventID': '759',
 'F1_Body_Significant_Strikes_Attempts': '20',
 'F1_Body_Significant_Strikes_Landed': '17',
 'F1_Body_Total_Strikes_Attempts': '20',
 'F1_Body_Total_Strikes_Landed': '17',
 'F1_Clinch_Body_Strikes_Attempts': '5',
 'F1_Clinch_Body_Strikes_Landed': '5',
 'F1_Clinch_Head_Strikes_Attempts': '11',
 'F1_Clinch_Head_Strikes_Landed': '8',
 'F1_Clinch_Leg_Strikes_Attempts': '0',
 'F1_Clinch_Leg_Strikes_Landed': '0',
 'F1_Clinch_Significant_Strikes_Attempts': '16',
 'F1_Clinch_Significant_Strikes_Landed': '13',
 'F1_Clinch_Total_Strikes_Attempts': '20',
 'F1_Clinch_Total_Strikes_Landed': '17',
 'F1_Distance_Body_Strikes_Attempts': '15',
 'F1_Distance_Body_Strikes_Landed': '12',
 'F1_Distance_Head_Strikes_Attempts': '69',
 'F1_Distance_Head_Strikes_Landed': '21',
 'F1_Distance_Leg_Strikes_Attempts': '9',
 'F1_Distance_Leg_Strikes_Landed': '6',
 'F1_Distance_Strikes_Attempts': '93',
 'F1_Distance_Strikes_Landed': '39',
 'F1_FighterID': '2266'

In [45]:
df = pd.DataFrame([clean_dict])

In [47]:
df.dtypes

Accolade                                  object
CurrentRnd                                object
EventID                                   object
F1_Body_Significant_Strikes_Attempts      object
F1_Body_Significant_Strikes_Landed        object
F1_Body_Total_Strikes_Attempts            object
F1_Body_Total_Strikes_Landed              object
F1_Clinch_Body_Strikes_Attempts           object
F1_Clinch_Body_Strikes_Landed             object
F1_Clinch_Head_Strikes_Attempts           object
F1_Clinch_Head_Strikes_Landed             object
F1_Clinch_Leg_Strikes_Attempts            object
F1_Clinch_Leg_Strikes_Landed              object
F1_Clinch_Significant_Strikes_Attempts    object
F1_Clinch_Significant_Strikes_Landed      object
F1_Clinch_Total_Strikes_Attempts          object
F1_Clinch_Total_Strikes_Landed            object
F1_Distance_Body_Strikes_Attempts         object
F1_Distance_Body_Strikes_Landed           object
F1_Distance_Head_Strikes_Attempts         object
F1_Distance_Head_Str

In [ ]:
if thing in some_list: some_list.remove(thing)